In [2]:
! pip install --upgrade openai==1.3.5
! pip install python-dotenv

In [3]:
import os
import re
import json
import time
import openai
import dotenv
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
from tqdm import tqdm

# 1. Load API key từ file key.env
load_dotenv("key.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [12]:
# 2. Hàm gọi GPT để gán label cho một record
def extract_json_list(text):
    """
    Tìm và trả về JSON list đầu tiên trong chuỗi (ngay cả khi có bọc ```json)
    """
    # Nếu được bọc trong ```json ``` thì loại bỏ
    match = re.search(r"```(?:json)?\s*(\[.*?\])\s*```", text, re.DOTALL)
    if match:
        return match.group(1)

    # Hoặc tìm JSON list thuần trong văn bản
    match = re.search(r"(\[.*?\])", text, re.DOTALL)
    if match:
        return match.group(1)

    raise ValueError("Không tìm thấy JSON list trong content GPT trả về.")

def label_options(record, model="gpt-4o", temperature=0):
    context = record.get("context", "")
    question = record.get("question", "")
    options = record.get("options", [])
    options = [opt.strip() for opt in options]
    options = ", ".join(options)
    prompt = f"""
    You are an AI assistant tasked with classifying the correctness of time-related answer options.
    
    Example:
    "context": "Một nhóm bạn quyết định tổ chức một sự kiện kỹ thuật số với nhiều hoạt động thú vị và các phiên thảo luận trực tuyến. Họ lên kế hoạch chuẩn bị nội dung, tìm diễn giả và quảng bá sự kiện cho đến khi tất cả mọi thứ sẵn sàng.",
    "question": "Mất bao lâu để chuẩn bị cho một sự kiện kỹ thuật số?",
    "options": ["3 tuần","2 tuần","5 tháng","1 năm"],
    "labels": ["yes","yes","no","no"]

    "context": "Một nhóm bạn quyết định tổ chức một buổi tiệc tùng để kỷ niệm thành công của dự án nghệ thuật mà họ đã thực hiện trong suốt thời gian qua. Họ muốn tạo ra một không gian vui vẻ và đầy sáng tạo, nơi mọi người có thể thưởng thức âm nhạc và nghệ thuật đa dạng.",
    "question": "Mất bao lâu để tổ chức một buổi tiệc tùng cho nhóm bạn?",
    "options": ["3 tuần", "6 tuần", "2 tháng", "5 ngày"],
    "labels": ["yes", "yes", "yes", "yes"]

    Given the context: \"{context}\", and the question: \"{question}\", evaluate whether each option in {options} correctly describes a duration.

    Return a JSON list called "labels" with the same length as "options", where each element is:
    - "yes" if the option correctly reflects the duration implied by the context and question,
    - "no" otherwise.

    Constraints:
    - Only return the list of labels. Do not include any additional text, explanation, or formatting.
    """


    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Bạn là trợ lý AI chuyên gán label yes/no."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=100,
    )

    content = response.choices[0].message.content.strip()
    # print(f"Prompt: {prompt}")
    # print(f"Content: {content}")

    try:
        # Trích xuất phần JSON list hợp lệ
        json_text = extract_json_list(content)
        labels = json.loads(json_text)
        assert isinstance(labels, list)
        return labels
    except Exception as e:
        print(f"[LỖI PARSE JSON] Content không hợp lệ:\n{content}")
        raise e


In [13]:
import tqdm
def main(input_path="public_test.txt", output_path="public_test_gpt_4o_ver3.txt"):
    with open(input_path, "r", encoding="utf-8") as fin, \
         open(output_path, "w", encoding="utf-8") as fout:
        for line in tqdm.tqdm(fin.readlines()):
            line = line.strip()
            if not line:
                continue
            record = json.loads(line)
            # Gọi GPT gán labels
            labels = label_options(record)
            # except Exception as e:
            #     print(f"Error when labeling QID={record.get('qid')}: {e}")
                # nếu lỗi, có thể thử chờ rồi gọi lại
            time.sleep(5)
            labels = label_options(record)
            # Gắn thêm trường labels
            record["labels"] = labels
            # Ghi ra file (mỗi dòng 1 JSON)
            fout.write(json.dumps(record, ensure_ascii=False) + "\n")
            # tránh call quá nhanh
            time.sleep(0.5)

In [14]:
if __name__ == "__main__":
    main()
    print("Hoàn thành!")

  0%|          | 0/400 [00:00<?, ?it/s]

 44%|████▍     | 177/400 [1:42:09<2:08:42, 34.63s/it]


KeyboardInterrupt: 